In [3]:
from transformers import AdamW, get_scheduler
from torch.utils.data import DataLoader
from vncorenlp import VnCoreNLP
from tqdm.auto import tqdm
import numpy as np
import torch
import random
from datasets import load_dataset


import nbimporter
from utlis import *
from preprocessing import preprocess
from models import CustomXLMModel
from metrics import *
from loss import *

In [ ]:
# Set Seed
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True


# Load datasets
trainset_path, devset_path = get_train_dev_path()
data_files = {'train': trainset_path, 
              'dev': devset_path}

dataset = load_dataset('csv', data_files=data_files)

# Preprocess
preprocess = preprocess()
tokenized_datasets = preprocess.run(dataset)

train_dataloader = DataLoader(tokenized_datasets["train"], 
                              batch_size=32, 
                              shuffle=True)

dev_dataloader = DataLoader(tokenized_datasets["dev"], 
                             batch_size=32)

# Model 
model = CustomXLMModel()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 1
num_training_steps = num_epochs*len(train_dataloader)
lr_scheduler = get_scheduler(
    'linear',
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

# Training
pb_train = tqdm(range(num_training_steps))
pb_dev = tqdm(range(num_epochs*len(dev_dataloader)))
best_score = -1

for epoch in range(num_epochs):
    train_loss = 0
    val_loss = 0
    
    # Train
    model.train()
    for batch in train_dataloader:
        inputs = {'input_ids': batch['input_ids'].to(device),
                  'attention_mask': batch['attention_mask'].to(device)}
        outputs_classifier, outputs_regressor = model(**inputs)
        print(f"C:{outputs_classifier}, R:{outputs_regressor}")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating dev split: 0 examples [00:00, ? examples/s]

C:\Users\lemai\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\lemai\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Map:   0%|          | 0/2951 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/737 [00:00<?, ? examples/s]

Map:   0%|          | 0/2951 [00:00<?, ? examples/s]

Map:   0%|          | 0/737 [00:00<?, ? examples/s]

Map:   0%|          | 0/2951 [00:00<?, ? examples/s]

Map:   0%|          | 0/737 [00:00<?, ? examples/s]